In [1]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 4.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import os
import csv
from datetime import datetime
from catboost import CatBoostRegressor
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Models Estimation

In [31]:
path = '/content/drive/MyDrive/Virtual env/z_inhomogenous_modeling/ecoflex_181221_rf_rd_data/'
path1 = '/content/drive/MyDrive/Virtual env/z_inhomogenous_modeling/x_y_data_all_points/'
print(len(sorted(os.listdir(path))))
rnd_idx = np.array([2,8,27,32,33,97,104,105,109,116,131,136,143,148,154,158,164,170,186,199])

400


In [32]:
arr = np.array(pd.read_csv(path1+f'{rnd_idx[0]}_x_y_pos_vel_ecoflex_181221_data.csv'))[:,:2]
arr.shape

(27234, 2)

In [33]:
y = np.array(pd.read_csv(path+f'{rnd_idx[0]}_force_ecoflex_181221_derivative_data.csv'))
x2 = np.array(pd.read_csv(path+f'{rnd_idx[0]}_pos_ecoflex_181221_derivative_data.csv'))
x2.shape, y.shape

((27234, 10), (27234, 1))

In [34]:
x = np.concatenate([x2,arr], axis = 1)
x.shape

(27234, 12)

In [35]:
for i in range(len(rnd_idx)-1):
    arr = np.array(pd.read_csv(path1+f'{rnd_idx[i+1]}_x_y_pos_vel_ecoflex_181221_data.csv'))[:,:2] # x and y values
    x1 = np.array(pd.read_csv(path+f'{rnd_idx[i+1]}_pos_ecoflex_181221_derivative_data.csv')) # fractional derivatives
    y1 = np.array(pd.read_csv(path+f'{rnd_idx[i+1]}_force_ecoflex_181221_derivative_data.csv'))
    x2 = np.concatenate([x1,arr], axis = 1)
    x = np.concatenate([x,x2])
    y = np.concatenate([y,y1])
print(x.shape, y.shape)

(544739, 12) (544739, 1)


In [36]:
y_test = np.array(pd.read_csv(path+f'{rnd_idx[0]-1}_force_ecoflex_181221_derivative_data.csv'))
x_t1 = np.array(pd.read_csv(path+f'{rnd_idx[0]-1}_pos_ecoflex_181221_derivative_data.csv'))
arr = np.array(pd.read_csv(path1+f'{rnd_idx[0]-1}_x_y_pos_vel_ecoflex_181221_data.csv'))[:,:2]
x_test = np.concatenate([x_t1,arr], axis = 1)

x_test.shape, y_test.shape

((27237, 12), (27237, 1))

In [37]:
for i in range(len(rnd_idx)//2):
    arr = np.array(pd.read_csv(path1+f'{rnd_idx[i+1]-1}_x_y_pos_vel_ecoflex_181221_data.csv'))[:,:2] # x and y values
    x1 = np.array(pd.read_csv(path+f'{rnd_idx[i+1]-1}_pos_ecoflex_181221_derivative_data.csv')) # fractional derivatives
    y1 = np.array(pd.read_csv(path+f'{rnd_idx[i+1]-1}_force_ecoflex_181221_derivative_data.csv'))
    x2 = np.concatenate([x1,arr], axis = 1)
    x_test = np.concatenate([x_test,x2])
    y_test = np.concatenate([y_test,y1])
print(x_test.shape, y_test.shape)

(299599, 12) (299599, 1)


In [38]:
cat = CatBoostRegressor()
cat.fit(x,y)
y_pred = cat.predict(x_test)

Learning rate set to 0.110789
0:	learn: 3.6550338	total: 73.3ms	remaining: 1m 13s
1:	learn: 3.2666911	total: 125ms	remaining: 1m 2s
2:	learn: 2.9225150	total: 183ms	remaining: 1m
3:	learn: 2.6139294	total: 239ms	remaining: 59.5s
4:	learn: 2.3414661	total: 292ms	remaining: 58.2s
5:	learn: 2.0975956	total: 343ms	remaining: 56.9s
6:	learn: 1.8791707	total: 393ms	remaining: 55.7s
7:	learn: 1.6866113	total: 443ms	remaining: 54.9s
8:	learn: 1.5160143	total: 491ms	remaining: 54.1s
9:	learn: 1.3618454	total: 561ms	remaining: 55.5s
10:	learn: 1.2251896	total: 610ms	remaining: 54.9s
11:	learn: 1.1025102	total: 665ms	remaining: 54.7s
12:	learn: 0.9919245	total: 716ms	remaining: 54.3s
13:	learn: 0.8939342	total: 769ms	remaining: 54.2s
14:	learn: 0.8073061	total: 819ms	remaining: 53.8s
15:	learn: 0.7305662	total: 872ms	remaining: 53.6s
16:	learn: 0.6625009	total: 924ms	remaining: 53.4s
17:	learn: 0.6027131	total: 976ms	remaining: 53.2s
18:	learn: 0.5499921	total: 1.03s	remaining: 53.1s
19:	learn: 0

In [39]:
print('RMSE:', mean_squared_error(y_pred, y_test)**0.5)

RMSE: 0.3191674665347111


In [43]:
len(rnd_idx)

20

In [40]:
for i in range(74,82):
    arr = np.array(pd.read_csv(path1+f'{i}_x_y_pos_vel_ecoflex_181221_data.csv'))[:,:2] # x and y values
    x1 = np.array(pd.read_csv(path+f'{i}_pos_ecoflex_181221_derivative_data.csv')) # fractional derivatives
    y1 = np.array(pd.read_csv(path+f'{i}_force_ecoflex_181221_derivative_data.csv'))
    x2 = np.concatenate([x1,arr], axis = 1)
    y_pred = cat.predict(x2)
    print(f'For point: {i}, Error: {mean_squared_error(y_pred, y1)**0.5}')

For point: 74, Error: 0.276156908202077
For point: 75, Error: 0.31030117170607013
For point: 76, Error: 0.16136058095534114
For point: 77, Error: 0.3674211558021242
For point: 78, Error: 0.40311284820728654
For point: 79, Error: 0.14337069141972394
For point: 80, Error: 0.19171325202380457
For point: 81, Error: 0.4264630653261238


In [41]:
error = []
for i in range(1,201):
    arr = np.array(pd.read_csv(path1+f'{i}_x_y_pos_vel_ecoflex_181221_data.csv'))[:,:2] # x and y values
    x1 = np.array(pd.read_csv(path+f'{i}_pos_ecoflex_181221_derivative_data.csv')) # fractional derivatives
    y1 = np.array(pd.read_csv(path+f'{i}_force_ecoflex_181221_derivative_data.csv'))
    x2 = np.concatenate([x1,arr], axis = 1)
    y_pred = cat.predict(x2)
    statement = f'For point: {i}, Error: {mean_squared_error(y_pred, y1)**0.5}\n'
    #print(f'For point: {i}, Error: {mean_squared_error(y_pred, y1)**0.5}')
    error.append(mean_squared_error(y_pred, y1)**0.5)
    with open('resuts.txt','a') as f:
        f.write(statement)
print(sum(error)/len(error))

0.24181764215365706


In [42]:
max(error)

0.8977122825061673